In [20]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
import string
import catboost

Откроем файл с данными.

In [21]:
df = pd.read_csv('credit_train.csv')

In [22]:
df.head()

,id,gender,age,marital_status,credit_sum,credit_month,tariff_id,education,credit_count,overdue_credit_count,job_pos,credit_score,region,income,opened
0,26031,M,24,UNM,"24000,00",10,1.60,SCH,2.0,0.0,SPC,"0,728851",ОБЛ ЛЕНИНГРАДСКАЯ,35000.0,True
1,148877,M,23,UNM,"16339,00",18,1.10,SCH,2.0,0.0,SPC,"0,347385",МОСКВА Г,38000.0,True
2,142019,M,54,UNM,"15652,00",10,1.32,GRD,2.0,0.0,SPC,"0,349705",ОБЛ МОСКОВСКАЯ,38000.0,True
3,146816,F,52,UNM,"16560,00",10,1.10,SCH,1.0,0.0,SPC,"0,359232",РЕСП МОРДОВИЯ,28000.0,False
4,27699,F,63,WID,"9078,00",18,1.10,SCH,4.0,0.0,SPC,"0,529368",ЧЕЛЯБИНСКАЯ ОБЛ,30000.0,False


In [23]:
df.shape

(114399, 15)

Форматирование данных

In [24]:
df['education'] = df['education'].replace('SCH', '1')
df['education'] = df['education'].replace('GRD', '2')
df['education'] = df['education'].replace('UGR', '3')
df['education'] = df['education'].replace('PGR', '4')
df['education'] = df['education'].replace('ACD', '5')
df['marital_status'] = df['marital_status'].replace('UNM', '1')
df['marital_status'] = df['marital_status'].replace('WID', '2')
df['marital_status'] = df['marital_status'].replace('MAR', '3')
df['marital_status'] = df['marital_status'].replace('DIV', '4')
df['marital_status'] = df['marital_status'].replace('CIV', '5')
df['job_pos'] = df['job_pos'].replace('SPC', '1')
df['job_pos'] = df['job_pos'].replace('UMN', '2')
df['job_pos'] = df['job_pos'].replace('ATP', '3')
df['job_pos'] = df['job_pos'].replace('BIS', '4')
df['job_pos'] = df['job_pos'].replace('DIR', '5')
df['job_pos'] = df['job_pos'].replace('PNA', '6')
df['job_pos'] = df['job_pos'].replace('WRP', '7')
df['job_pos'] = df['job_pos'].replace('WRK', '8')
df['job_pos'] = df['job_pos'].replace('NOR', '9')
df['job_pos'] = df['job_pos'].replace('BIU', '10')
df['job_pos'] = df['job_pos'].replace('WOI', '11')
df['job_pos'] = df['job_pos'].replace('INP', '12')
df['job_pos'] = df['job_pos'].replace('HSK', '13')
df['job_pos'] = df['job_pos'].replace('PNV', '14')
df['job_pos'] = df['job_pos'].replace('PNI', '15')
df['job_pos'] = df['job_pos'].replace('PNS', '16')
df['job_pos'] = df['job_pos'].replace('ONB', '17')
df['job_pos'] = df['job_pos'].replace('INV', '18')
y_train = df['opened']
X_train = df.drop(columns=['gender', 'region', 'opened'])#'marital_status', 'education', 'job_pos'
X_train['credit_sum'] = X_train['credit_sum'].apply(lambda x: float(x.replace(',', '.')))
X_train['credit_score'] = X_train['credit_score'].apply(lambda x: float(x.replace(',', '.')))
X_train['education'] = X_train['education'].apply(lambda x: int(x))
X_train['job_pos'] = X_train['job_pos'].apply(lambda x: int(x))
X_train['marital_status'] = X_train['marital_status'].apply(lambda x: int(x))
X_train['credit_score'] = X_train['credit_score'].apply(lambda x: int(x))
X_train = X_train.fillna(-1)

In [25]:
X_train

,id,age,marital_status,credit_sum,credit_month,tariff_id,education,credit_count,overdue_credit_count,job_pos,credit_score,income
0,26031,24,1,24000.0,10,1.60,1,2.0,0.0,1,0,35000.0
1,148877,23,1,16339.0,18,1.10,1,2.0,0.0,1,0,38000.0
2,142019,54,1,15652.0,10,1.32,2,2.0,0.0,1,0,38000.0
3,146816,52,1,16560.0,10,1.10,1,1.0,0.0,1,0,28000.0
4,27699,63,2,9078.0,18,1.10,1,4.0,0.0,1,0,30000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
114394,119880,29,3,29678.0,10,1.60,2,1.0,0.0,1,0,25000.0
114395,103695,29,3,23068.0,15,1.10,2,3.0,0.0,1,0,30000.0
114396,131933,45,3,16418.0,10,1.10,1,1.0,0.0,2,0,30400.0
114397,146868,49,3,12169.0,10,1.40,1,1.0,0.0,1,0,16000.0


Обучим модель

In [ ]:
clf = RandomForestClassifier(n_estimators=31, max_depth=31)
print(cross_val_score(clf, X_train, y_train, cv=5, scoring='f1').mean())
clf.fit(X_train, y_train)

In [ ]:
'''lgbm = lightgbm.LGBMClassifier(n_estimators=1000, max_depth=400)
print(cross_val_score(lgbm, X_train, y_train, scoring='f1').mean())
lgbm.fit(X_train, y_train)'''

In [27]:
from catboost import CatBoostClassifier, Pool


train_data = Pool(X_train, y_train)
model = CatBoostClassifier(iterations=1000, max_depth=12, verbose=False)
model.fit(train_data)

model.save_model("model")
from_file = CatBoostClassifier()
from_file.load_model("model")

Считаем тестовую выборку и предскажем ответы для нее

In [28]:
df_test = pd.read_csv('credit_test.csv')
df_test['education'] = df_test['education'].replace('SCH', '1')
df_test['education'] = df_test['education'].replace('GRD', '2')
df_test['education'] = df_test['education'].replace('UGR', '3')
df_test['education'] = df_test['education'].replace('PGR', '4')
df_test['education'] = df_test['education'].replace('ACD', '5')
df_test['marital_status'] = df_test['marital_status'].replace('UNM', '1')
df_test['marital_status'] = df_test['marital_status'].replace('WID', '2')
df_test['marital_status'] = df_test['marital_status'].replace('MAR', '3')
df_test['marital_status'] = df_test['marital_status'].replace('DIV', '4')
df_test['marital_status'] = df_test['marital_status'].replace('CIV', '5')
df_test['job_pos'] = df_test['job_pos'].replace('SPC', '1')
df_test['job_pos'] = df_test['job_pos'].replace('UMN', '2')
df_test['job_pos'] = df_test['job_pos'].replace('ATP', '3')
df_test['job_pos'] = df_test['job_pos'].replace('BIS', '4')
df_test['job_pos'] = df_test['job_pos'].replace('DIR', '5')
df_test['job_pos'] = df_test['job_pos'].replace('PNA', '6')
df_test['job_pos'] = df_test['job_pos'].replace('WRP', '7')
df_test['job_pos'] = df_test['job_pos'].replace('WRK', '8')
df_test['job_pos'] = df_test['job_pos'].replace('NOR', '9')
df_test['job_pos'] = df_test['job_pos'].replace('BIU', '10')
df_test['job_pos'] = df_test['job_pos'].replace('WOI', '11')
df_test['job_pos'] = df_test['job_pos'].replace('INP', '12')
df_test['job_pos'] = df_test['job_pos'].replace('HSK', '13')
df_test['job_pos'] = df_test['job_pos'].replace('PNV', '14')
df_test['job_pos'] = df_test['job_pos'].replace('PNI', '15')
df_test['job_pos'] = df_test['job_pos'].replace('PNS', '16')
df_test['job_pos'] = df_test['job_pos'].replace('ONB', '17')
df_test['job_pos'] = df_test['job_pos'].replace('INV', '18')
df_test.head()


,id,gender,age,marital_status,credit_sum,credit_month,tariff_id,education,credit_count,overdue_credit_count,job_pos,credit_score,region,income
0,39162,F,18,1,"39469,00",6,1.44,2,NaN,NaN,1,"0,424628",САНКТ-ПЕТЕРБУРГ Г,30000.0
1,10673,F,63,2,"16693,00",10,1.32,2,2.0,0.0,1,"0,311945",ОБЛ ЛЕНИНГРАДСКАЯ,21250.0
2,121409,M,42,3,"60260,00",10,1.10,2,5.0,0.0,1,"0,421982",МОСКВА Г,150000.0
3,23504,M,41,3,"32479,00",12,1.10,2,1.0,0.0,5,"0,406572",ТАТАРСТАН РЕСП,150000.0
4,69269,F,24,1,"8159,00",12,1.10,1,2.0,0.0,1,"0,418243",ТВЕРСКАЯ ОБЛ,16000.0


Удалим колонки с которыми мы не работаем, как и на тренировочной выборке

In [29]:
X_test = df_test.drop(columns=['gender', 'region'])#'marital_status', 'education', 'job_pos'
X_test['credit_sum'] = X_test['credit_sum'].apply(lambda x: float(x.replace(',', '.')))
X_test['credit_score'] = X_test['credit_score'].apply(lambda x: float(x.replace(',', '.')))
X_test['education'] = X_test['education'].apply(lambda x: int(x))
X_test['job_pos'] = X_test['job_pos'].apply(lambda x: int(x))
X_test['marital_status'] = X_test['marital_status'].apply(lambda x: int(x))
X_test['credit_score'] = X_test['credit_score'].apply(lambda x: int(x))
X_test = X_test.fillna(-1)

Предскажем значения для тестовой выборки

In [30]:
y_pred = model.predict(X_test)#aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

In [31]:
y_pred

array(['False', 'False', 'False', ..., 'False', 'False', 'False'],
      dtype=object)

In [32]:
y_pred

array(['False', 'False', 'False', ..., 'False', 'False', 'False'],
      dtype=object)

Создадим отдельный датасет для отправки на kaggle. 

Данный датасет будет содержать только два необходимых столбца — id и наш ответ.

In [33]:
df_kaggle = df_test[['id',]]
df_kaggle['opened'] = y_pred.astype(bool)
df_kaggle.head()

C:\Users\Никита\AppData\Local\Temp\ipykernel_10216\1716018407.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kaggle['opened'] = y_pred.astype(bool)


,id,opened
0,39162,True
1,10673,True
2,121409,True
3,23504,True
4,69269,True


Сохраним полученный результат в .csv файл для отправки на kaggle

In [19]:
df_kaggle.to_csv('my_submission.csv', index=False)